In [2]:

import os
import cv2
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Lambda, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from PIL import Image



c:\Users\hiba\Desktop\indus\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
DATASET_PATH = "eleves/"

# Créer un dataframe pour stocker les informations des images
def create_dataset(dataset_path):
    images = []
    labels = []
    
    for person_name in os.listdir(dataset_path):
        person_path = os.path.join(dataset_path, person_name)
        
        if os.path.isdir(person_path):
            for image_name in os.listdir(person_path):
                image_path = os.path.join(person_path, image_name)
                
                # Vérifier que c'est bien un fichier image
                if image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                    images.append(image_path)
                    labels.append(person_name)
    
    return pd.DataFrame({'image_path': images, 'label': labels})

df = create_dataset(DATASET_PATH)
print(f"Dataset créé avec {len(df)} images")
print(df.head())


Dataset créé avec 117 images
                                         image_path              label
0  eleves/aissa_lamkharbach\IMG-20250423-WA0052.jpg  aissa_lamkharbach
1  eleves/aissa_lamkharbach\IMG-20250423-WA0053.jpg  aissa_lamkharbach
2    eleves/Fatima_elfadili\IMG-20250423-WA0001.jpg    Fatima_elfadili
3    eleves/Fatima_elfadili\IMG-20250423-WA0002.jpg    Fatima_elfadili
4    eleves/Fatima_elfadili\IMG-20250423-WA0003.jpg    Fatima_elfadili


In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(keep_all=True, device=device)

# Fonction de prétraitement avec détection des visages
def preprocess_image(image_path, target_size=(160, 160)):
    try:
        # Charger l'image
        img = Image.open(image_path).convert('RGB')
        
        # Détection et alignement du visage
        boxes, probs = mtcnn.detect(img)
        
        if boxes is not None:
            # Prendre la boîte avec la plus haute probabilité
            box = boxes[np.argmax(probs)]
            
            # Recadrer le visage
            face = img.crop(box)
            face = face.resize(target_size)
            
            # Convertir en numpy array et normaliser
            face_array = np.array(face).astype('float32')
            face_array = (face_array - 127.5) / 128.0
            
            return face_array
        else:
            return None
    except Exception as e:
        print(f"Erreur lors du traitement de {image_path}: {e}")
        return None

# %%
# Appliquer le prétraitement à toutes les images
processed_images = []
valid_labels = []

for idx, row in df.iterrows():
    processed_img = preprocess_image(row['image_path'])
    if processed_img is not None:
        processed_images.append(processed_img)
        valid_labels.append(row['label'])

# Convertir en numpy arrays
X = np.array(processed_images)
y = np.array(valid_labels)

# Encoder les labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Sauvegarder le label encoder pour plus tard
np.save('label_encoder.npy', label_encoder.classes_)


In [5]:
def load_facenet_model():
    # Utiliser la version PyTorch de FaceNet
    resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)
    return resnet

facenet = load_facenet_model()

# Fonction pour extraire les embeddings
def get_embedding(face_pixels, model):
    # Convertir en tensor PyTorch
    face_tensor = torch.tensor(face_pixels.transpose(2, 0, 1)).unsqueeze(0).float().to(device)
    
    # Générer l'embedding
    with torch.no_grad():
        embedding = model(face_tensor)
    
    return embedding.cpu().numpy().reshape(-1)

# Générer les embeddings pour toutes les images
embeddings = np.array([get_embedding(face, facenet) for face in X])


In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    embeddings, y_encoded, test_size=0.2, random_state=42)

# Créer un simple classifieur (SVM serait meilleur en production)
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

classifier = SVC(kernel='linear', probability=True)
classifier.fit(X_train, y_train)

# Évaluer le classifieur
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy du classifieur: {accuracy:.2f}")

# Sauvegarder le classifieur
import joblib
joblib.dump(classifier, 'face_classifier.pkl')



Accuracy du classifieur: 0.96


['face_classifier.pkl']

In [16]:
import pyttsx3
import time

# Initialiser le moteur de synthèse vocale
engine = pyttsx3.init()
engine.setProperty('rate', 150)  # Vitesse de parole

# Configuration du temps de détection
DETECTION_DURATION = 30  # 3 minutes en secondes
start_time = datetime.now()

cap = cv2.VideoCapture(0)

# Créer le fichier CSV pour enregistrer les présences
attendance_csv = "attendance.csv"
attendance_df = pd.DataFrame(columns=['Nom', 'Prénom', 'Présent', 'Heure_detection'])

# Dictionnaire pour suivre les détections
detection_history = {label: {'detected': False, 
                            'time': None, 
                            'last_detection_time': None, 
                            'announced': False} 
                    for label in label_encoder.classes_}

# Message de bienvenue
engine.say("Bienvenue les élèves")
engine.runAndWait()

print("Démarrage de la reconnaissance faciale pour 3 minutes...")

try:
    while True:
        # Vérifier si le temps écoulé dépasse 3 minutes
        elapsed_time = (datetime.now() - start_time).total_seconds()
        if elapsed_time > DETECTION_DURATION:
            print("Temps de détection écoulé (3 minutes)")
            # Message de fin
            engine.say("Malheureusement, le temps est terminé")
            engine.runAndWait()
            break
        
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convertir l'image pour MTCNN
        img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        
        # Détection des visages
        boxes, probs = mtcnn.detect(img)
        
        if boxes is not None:
            for box in boxes:
                # Recadrer le visage
                face = img.crop(box)
                face = face.resize((160, 160))
                
                # Prétraitement
                face_array = np.array(face).astype('float32')
                face_array = (face_array - 127.5) / 128.0
                
                # Générer l'embedding
                embedding = get_embedding(face_array, facenet)
                
                # Prédire la personne
                proba = classifier.predict_proba(embedding.reshape(1, -1))
                max_proba = np.max(proba)
                
                if max_proba > 0.7:  # Seuil de confiance
                    pred_label = label_encoder.inverse_transform([np.argmax(proba)])[0]
                    current_time = datetime.now()
                    
                    # Mettre à jour le temps de dernière détection
                    detection_history[pred_label]['last_detection_time'] = current_time
                    
                    # Si c'est une nouvelle détection
                    if not detection_history[pred_label]['detected']:
                        detection_history[pred_label]['detected'] = True
                        detection_history[pred_label]['time'] = current_time
                    
                    # Vérifier si 2 secondes se sont écoulées depuis la détection initiale
                    if (not detection_history[pred_label]['announced'] and 
                        (current_time - detection_history[pred_label]['time']).total_seconds() >= 2):
                        nom, prenom = pred_label.split('_')
                        engine.say(f"Merci {nom} {prenom}")
                        engine.runAndWait()
                        detection_history[pred_label]['announced'] = True
                    
                    # Dessiner le rectangle et le nom
                    box = box.astype(int)
                    cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
                    cv2.putText(frame, pred_label, (box[0], box[1]-10), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                else:
                    # Visage inconnu détecté
                    box = box.astype(int)
                    cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 0, 255), 2)
                    cv2.putText(frame, "Inconnu", (box[0], box[1]-10), 
                              cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
                    
                
        
        # Afficher le résultat et le temps restant
        remaining_time = max(0, DETECTION_DURATION - elapsed_time)
        time_text = f"Temps restant: {int(remaining_time // 60)}:{int(remaining_time % 60):02d}"
        cv2.putText(frame, time_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.imshow('Reconnaissance Faciale', frame)
        
        # Quitter avec la touche 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    # Finaliser l'enregistrement des présences
    current_time = datetime.now()
    
    # Créer le dataframe final
    final_attendance = []
    
    for label in detection_history:
        nom, prenom = label.split('_')
        if detection_history[label]['detected']:
            status = "Présent"
            detection_time = detection_history[label]['time']
        else:
            status = "Absent"
            detection_time = current_time
        
        final_attendance.append({
            'Nom': nom,
            'Prénom': prenom,
            'Présent': status,
            'Heure_detection': detection_time.strftime("%Y-%m-%d %H:%M:%S")
        })
    
    # Créer le DataFrame final
    attendance_df = pd.DataFrame(final_attendance)
    
    # Sauvegarder le fichier CSV
    attendance_df.to_csv(attendance_csv, index=False)
    cap.release()
    cv2.destroyAllWindows()
    print("Détection terminée. Fichier de présence sauvegardé.")

Démarrage de la reconnaissance faciale pour 3 minutes...
Temps de détection écoulé (3 minutes)
Détection terminée. Fichier de présence sauvegardé.
